In [53]:
import os
import glob
import pandas as pd
import re
from astropy.io import fits

def extract_columns_from_txt(target_path):
    """
    Processes all .txt files directly in the target folder, extracting columns based on IDL format.
    Args:
        target_path (str): Full path to the target folder.
    Returns:
        results (list): List of dicts with extracted columns for each file.
    """
    if not os.path.exists(target_path):
        print(f'Target directory not found: {target_path}')
        return []
    files = glob.glob(os.path.join(target_path, '*.txt'))
    results = []
    for file in files:
        try:
            df = pd.read_csv(file, delim_whitespace=True, skiprows=31, header=None)
        except Exception as e:
            print(f'Error reading {file}: {e}')
            continue
        if df.shape[1] >= 15:
            ra   = df.iloc[:, 1]   # float
            dec  = df.iloc[:, 2]   # float
            band = df.iloc[:, 6]   # float
            date = df.iloc[:, 12]  # string
            name = df.iloc[:, 14]  # string
            results.append({
                'file': file,
                'ra': ra,
                'dec': dec,
                'band': band,
                'date': date,
                'name': name
            })
        else:
            print(f'File {file} does not have expected number of columns.')
    return results

def get_fits_files(target_path, file_type='int'):
    """Return list of FITS files of a given type in the target folder."""
    pattern = f"*{'-int' if file_type == 'int' else '-unc'}*.fits"
    return glob.glob(os.path.join(target_path, pattern))

def parse_filename(filename):
    """Extract year, month, filter, etc. from filename using regex."""
    # Example pattern, adjust as needed
    match = re.search(r'(\d{2}[A-Za-z]{3})(\d{1,2})_.*?([A-Za-z0-9]+)?-int', filename)
    if match:
        year_month = match.group(1)
        day = match.group(2)
        filter_name = match.group(3) if match.group(3) else None
        return year_month, day, filter_name
    return None, None, None

def validate_fits_header(fits_path, txt_record):
    """Compare FITS header info to .txt record."""
    with fits.open(fits_path) as hdul:
        hdr = hdul[0].header
        date_obs = hdr.get('DATE-OBS')
        date = hdr.get('DATE')
        numfrms = hdr.get('NUMFRMS')
        # Compare with txt_record['date'] and txt_record['numfrms'] if present
        numfrms_match = ('numfrms' not in txt_record) or (numfrms == txt_record['numfrms'])
        return (date == txt_record['date']) and numfrms_match

def process_target_folder(target_path):
    """Process one target folder for all validation steps."""
    txt_files = glob.glob(os.path.join(target_path, '*.txt'))
    fits_files = get_fits_files(target_path, file_type='int')
    for txt_file in txt_files:
        # Parse .txt file for expected filenames, dates, filters, numfrms
        # For each FITS file, parse filename and header, compare to .txt record
        pass  # Implement logic here

# Main logic to process all targets in Physics Data
base_dir = "/Users/tmzyece25/Desktop/Academia/Physics Data"
target_folders = [f for f in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, f))]

# Main loop
for target in target_folders:
    target_path = os.path.join(base_dir, target)
    process_target_folder(target_path)

In [54]:
import numpy as np
def parse_txt_file(txt_file):
    """Parse a .txt file and return a list of records with expected FITS filename, date, filter, numfrms, etc."""
    records = []
    try:
        # Read .txt file, skipping header lines as before
        df = pd.read_csv(txt_file, delim_whitespace=True, skiprows=31, header=None, dtype=str)
        fname_col = df.shape[1] - 1
        for idx, row in df.iterrows():
            fname = str(row[fname_col]).strip()
            # Only process rows where fname ends with '.fits'
            if not fname.endswith('.fits'):
                continue
            record = {
                'ra': row[1],
                'dec': row[2],
                'band': row[6],
                'date': row[12],
                'fname': fname
                # Add more fields if needed, e.g. numfrms if present
            }
            records.append(record)
    except Exception as e:
        print(f'Error parsing {txt_file}: {e}')
    return records

def process_target_folder(target_path):
    """Process one target folder for all validation steps."""
    txt_files = glob.glob(os.path.join(target_path, '*.txt'))
    fits_files = get_fits_files(target_path, file_type='int')
    fits_files_dict = {os.path.basename(f): f for f in fits_files}
    for txt_file in txt_files:
        txt_records = parse_txt_file(txt_file)
        for rec in txt_records:
            fits_name = rec['fname']
            if fits_name not in fits_files_dict:
                print(f"No FITS file found for TXT record fname '{fits_name}' in {txt_file}")
                continue
            fits_path = fits_files_dict[fits_name]
            # Validate FITS header against .txt record
            valid = validate_fits_header(fits_path, rec)
            if not valid:
                print(f"Header mismatch for FITS file {fits_name} and record in {txt_file}")
            else:
                print(f"Match: {fits_name} <-> TXT record (date: {rec['date']})")

In [55]:
import matplotlib.pyplot as plt
def visualize_matches(matches):
    """Visualize distribution of matched FITS files by date and band."""
    if not matches:
        print("No matches to visualize.")
        return
    dates = [m['date'] for m in matches]
    bands = [m['band'] for m in matches]
    plt.figure(figsize=(10,5))
    plt.hist(dates, bins=20, color='skyblue', edgecolor='black')
    plt.title('Distribution of FITS File Dates')
    plt.xlabel('Date')
    plt.ylabel('Count')
    plt.show()
    plt.figure(figsize=(8,4))
    plt.hist(bands, bins=len(set(bands)), color='salmon', edgecolor='black')
    plt.title('Distribution of Bands')
    plt.xlabel('Band')
    plt.ylabel('Count')
    plt.show()

# Example usage after collecting matches:
# visualize_matches(list_of_matched_records)

In [56]:
# Test code for FITS header and filename validation logic
import tempfile
import shutil

# Create a temporary test directory structure (simulate a Physics Data folder)
test_base_dir = tempfile.mkdtemp()
test_folder = os.path.join(test_base_dir, 'TestTarget')
os.makedirs(test_folder, exist_ok=True)

# Create a dummy .txt file with expected columns
txt_path = os.path.join(test_folder, 'test_data.txt')
with open(txt_path, 'w') as f:
    f.write('\n'*31)  # Simulate header lines
    f.write('0 10.0 20.0 0 0 0 1.0 0 0 0 0 0 2025-09-18 0 testfile-int.fits\n')

# Create a dummy FITS file with matching header
from astropy.io import fits
import numpy as np
fits_path = os.path.join(test_folder, 'testfile-int.fits')
hdu = fits.PrimaryHDU(np.zeros((10,10)))
hdu.header['DATE'] = '2025-09-18'
hdu.header['NUMFRMS'] = 0
hdu.writeto(fits_path, overwrite=True)

# Run your validation logic on the test folder
print('Running validation on test folder...')
process_target_folder(test_folder)

# Clean up test files after running
shutil.rmtree(test_base_dir)

Running validation on test folder...
Match: testfile-int.fits <-> TXT record (date: 2025-09-18)


/var/folders/pf/nh38xgvs0nndkbgjwfbcnmbm0000gp/T/ipykernel_76100/1460263817.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(txt_file, delim_whitespace=True, skiprows=31, header=None, dtype=str)


In [57]:
# Test your FITS and .txt validation code on your actual Physics Data directory
base_dir = "/Users/tmzyece25/Desktop/Academia/Physics Data"
print(f"Testing on base directory: {base_dir}")
target_folders = [f for f in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, f))]
for target in target_folders:
    target_path = os.path.join(base_dir, target)
    print(f"\nProcessing folder: {target_path}")
    process_target_folder(target_path)

Testing on base directory: /Users/tmzyece25/Desktop/Academia/Physics Data

Processing folder: /Users/tmzyece25/Desktop/Academia/Physics Data/01) 58.770302 -38.19514
No FITS file found for TXT record fname '01913r025-w1-int-1b.fits' in /Users/tmzyece25/Desktop/Academia/Physics Data/01) 58.770302 -38.19514/19JanICORE_0001_W1_query_used.tbl.txt
No FITS file found for TXT record fname '01915r048-w1-int-1b.fits' in /Users/tmzyece25/Desktop/Academia/Physics Data/01) 58.770302 -38.19514/19JanICORE_0001_W1_query_used.tbl.txt
No FITS file found for TXT record fname '01915r050-w1-int-1b.fits' in /Users/tmzyece25/Desktop/Academia/Physics Data/01) 58.770302 -38.19514/19JanICORE_0001_W1_query_used.tbl.txt
No FITS file found for TXT record fname '01917r024-w1-int-1b.fits' in /Users/tmzyece25/Desktop/Academia/Physics Data/01) 58.770302 -38.19514/19JanICORE_0001_W1_query_used.tbl.txt
No FITS file found for TXT record fname '01917r025-w1-int-1b.fits' in /Users/tmzyece25/Desktop/Academia/Physics Data/01

/var/folders/pf/nh38xgvs0nndkbgjwfbcnmbm0000gp/T/ipykernel_76100/1460263817.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(txt_file, delim_whitespace=True, skiprows=31, header=None, dtype=str)
/var/folders/pf/nh38xgvs0nndkbgjwfbcnmbm0000gp/T/ipykernel_76100/1460263817.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(txt_file, delim_whitespace=True, skiprows=31, header=None, dtype=str)
/var/folders/pf/nh38xgvs0nndkbgjwfbcnmbm0000gp/T/ipykernel_76100/1460263817.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(txt_file, delim_whitespace=True, skiprows=31, header=None, dtype=str)
/var/folders/pf/nh38xgvs0nndkbgjwfbcnmbm0000gp/T/ipykernel_76

No FITS file found for TXT record fname '51373b241-w2-int-1b.fits' in /Users/tmzyece25/Desktop/Academia/Physics Data/01) 58.770302 -38.19514/14Aug2.tbl.txt
No FITS file found for TXT record fname '51373b242-w2-int-1b.fits' in /Users/tmzyece25/Desktop/Academia/Physics Data/01) 58.770302 -38.19514/14Aug2.tbl.txt
No FITS file found for TXT record fname '51376a217-w2-int-1b.fits' in /Users/tmzyece25/Desktop/Academia/Physics Data/01) 58.770302 -38.19514/14Aug2.tbl.txt
No FITS file found for TXT record fname '51376a218-w2-int-1b.fits' in /Users/tmzyece25/Desktop/Academia/Physics Data/01) 58.770302 -38.19514/14Aug2.tbl.txt
No FITS file found for TXT record fname '51377b242-w2-int-1b.fits' in /Users/tmzyece25/Desktop/Academia/Physics Data/01) 58.770302 -38.19514/14Aug2.tbl.txt
No FITS file found for TXT record fname '51377b243-w2-int-1b.fits' in /Users/tmzyece25/Desktop/Academia/Physics Data/01) 58.770302 -38.19514/14Aug2.tbl.txt
No FITS file found for TXT record fname '51380a216-w2-int-1b.fit

/var/folders/pf/nh38xgvs0nndkbgjwfbcnmbm0000gp/T/ipykernel_76100/1460263817.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(txt_file, delim_whitespace=True, skiprows=31, header=None, dtype=str)
/var/folders/pf/nh38xgvs0nndkbgjwfbcnmbm0000gp/T/ipykernel_76100/1460263817.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(txt_file, delim_whitespace=True, skiprows=31, header=None, dtype=str)
/var/folders/pf/nh38xgvs0nndkbgjwfbcnmbm0000gp/T/ipykernel_76100/1460263817.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(txt_file, delim_whitespace=True, skiprows=31, header=None, dtype=str)
/var/folders/pf/nh38xgvs0nndkbgjwfbcnmbm0000gp/T/ipykernel_76

No FITS file found for TXT record fname '19386r229-w2-int-1b.fits' in /Users/tmzyece25/Desktop/Academia/Physics Data/01) 58.770302 -38.19514/20AugICORE_0001_W2_query_used.tbl.txt
No FITS file found for TXT record fname '19388r204-w2-int-1b.fits' in /Users/tmzyece25/Desktop/Academia/Physics Data/01) 58.770302 -38.19514/20AugICORE_0001_W2_query_used.tbl.txt
No FITS file found for TXT record fname '19390r228-w2-int-1b.fits' in /Users/tmzyece25/Desktop/Academia/Physics Data/01) 58.770302 -38.19514/20AugICORE_0001_W2_query_used.tbl.txt
No FITS file found for TXT record fname '19390r230-w2-int-1b.fits' in /Users/tmzyece25/Desktop/Academia/Physics Data/01) 58.770302 -38.19514/20AugICORE_0001_W2_query_used.tbl.txt
No FITS file found for TXT record fname '19392r204-w2-int-1b.fits' in /Users/tmzyece25/Desktop/Academia/Physics Data/01) 58.770302 -38.19514/20AugICORE_0001_W2_query_used.tbl.txt
No FITS file found for TXT record fname '19394r228-w2-int-1b.fits' in /Users/tmzyece25/Desktop/Academia/P

In [58]:
# Debug: Print base names of FITS files and TXT records for a selected folder
import pprint

def debug_base_names(target_path):
    txt_files = glob.glob(os.path.join(target_path, '*.txt'))
    fits_files = get_fits_files(target_path, file_type='int')
    def get_base_name(filename):
        base = filename
        for ext in ['.fits', '.txt']:
            if base.endswith(ext):
                base = base[:-len(ext)]
        for suf in ['-int', '-unc', '.tbl']:
            if base.endswith(suf):
                base = base[:-len(suf)]
        return base
    fits_basenames = [get_base_name(os.path.basename(f)) for f in fits_files]
    print('FITS base names:')
    pprint.pprint(fits_basenames)
    for txt_file in txt_files:
        print(f'\nTXT file: {txt_file}')
        txt_records = parse_txt_file(txt_file)
        txt_basenames = [get_base_name(rec['fname']) for rec in txt_records]
        print('TXT record base names:')
        pprint.pprint(txt_basenames)

# Example usage: debug a specific folder
debug_base_names('/Users/tmzyece25/Desktop/Academia/Physics Data/01) 58.770302 -38.19514')


FITS base names:
['23AugICORE_0001_W2_mosaic',
 '17AugICORE_0001_W1_mosaic',
 '20JanICORE_0001_W1_mosaic',
 '21AugICORE_0001_W1_mosaic',
 '22JanICORE_0001_W2_mosaic',
 '18AugICORE_0001_W2_mosaic',
 '19JanICORE_0001_W2_mosaic',
 '18JanICORE_0001_W1_mosaic',
 '19AugICORE_0001_W1_mosaic',
 '20AugICORE_0001_W2_mosaic',
 '17JanICORE_0001_W2_mosaic',
 '23JanICORE_0001_W1_mosaic',
 '24JanICORE_0001_W2_mosaic',
 '22AugICORE_0001_W1_mosaic',
 '21JanICORE_0001_W2_mosaic',
 '18AugICORE_0001_W1_mosaic',
 '19JanICORE_0001_W1_mosaic',
 '20JanICORE_0001_W2_mosaic',
 '23AugICORE_0001_W1_mosaic',
 '17AugICORE_0001_W2_mosaic',
 '22JanICORE_0001_W1_mosaic',
 '21AugICORE_0001_W2_mosaic',
 '24JanICORE_0001_W1_mosaic',
 '23JanICORE_0001_W2_mosaic',
 '20AugICORE_0001_W1_mosaic',
 '21JanICORE_0001_W1_mosaic',
 '16AugICORE_0001_W1_mosaic',
 '22AugICORE_0001_W2_mosaic',
 '18JanICORE_0001_W2_mosaic',
 '19AugICORE_0001_W2_mosaic']

TXT file: /Users/tmzyece25/Desktop/Academia/Physics Data/01) 58.770302 -38.19514/1

/var/folders/pf/nh38xgvs0nndkbgjwfbcnmbm0000gp/T/ipykernel_76100/1460263817.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(txt_file, delim_whitespace=True, skiprows=31, header=None, dtype=str)
/var/folders/pf/nh38xgvs0nndkbgjwfbcnmbm0000gp/T/ipykernel_76100/1460263817.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(txt_file, delim_whitespace=True, skiprows=31, header=None, dtype=str)
/var/folders/pf/nh38xgvs0nndkbgjwfbcnmbm0000gp/T/ipykernel_76100/1460263817.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(txt_file, delim_whitespace=True, skiprows=31, header=None, dtype=str)
/var/folders/pf/nh38xgvs0nndkbgjwfbcnmbm0000gp/T/ipykernel_76

TXT record base names:
['62373b239-w1-int-1b',
 '62377b238-w1-int-1b',
 '62377b239-w1-int-1b',
 '62380a215-w1-int-1b',
 '62384a213-w1-int-1b',
 '62384a214-w1-int-1b',
 '62396a213-w1-int-1b',
 '62396a214-w1-int-1b',
 '62397b238-w1-int-1b',
 '62397b239-w1-int-1b',
 '62400a213-w1-int-1b',
 '62400a214-w1-int-1b',
 '62400a215-w1-int-1b',
 '62401b239-w1-int-1b',
 '62404a213-w1-int-1b',
 '62404a214-w1-int-1b',
 '62405b238-w1-int-1b',
 '62405b239-w1-int-1b',
 '62408a213-w1-int-1b',
 '62408a214-w1-int-1b',
 '62409b238-w1-int-1b',
 '62409b239-w1-int-1b',
 '62413b238-w1-int-1b',
 '62413b239-w1-int-1b',
 '62416a213-w1-int-1b',
 '62416a214-w1-int-1b',
 '62425b238-w1-int-1b',
 '62425b239-w1-int-1b',
 '62428a213-w1-int-1b',
 '62428a214-w1-int-1b',
 '62429b238-w1-int-1b',
 '62429b239-w1-int-1b',
 '62432a213-w1-int-1b',
 '62432a214-w1-int-1b',
 '62433b238-w1-int-1b',
 '62436a214-w1-int-1b',
 '62436a215-w1-int-1b',
 '62437b238-w1-int-1b',
 '62440a213-w1-int-1b',
 '62440a214-w1-int-1b',
 '62516a210-w1-in

/var/folders/pf/nh38xgvs0nndkbgjwfbcnmbm0000gp/T/ipykernel_76100/1460263817.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(txt_file, delim_whitespace=True, skiprows=31, header=None, dtype=str)
/var/folders/pf/nh38xgvs0nndkbgjwfbcnmbm0000gp/T/ipykernel_76100/1460263817.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(txt_file, delim_whitespace=True, skiprows=31, header=None, dtype=str)
/var/folders/pf/nh38xgvs0nndkbgjwfbcnmbm0000gp/T/ipykernel_76100/1460263817.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(txt_file, delim_whitespace=True, skiprows=31, header=None, dtype=str)
/var/folders/pf/nh38xgvs0nndkbgjwfbcnmbm0000gp/T/ipykernel_76

TXT record base names:
['89663a050-w1-int-1b',
 '89665a025-w1-int-1b',
 '89673a025-w1-int-1b',
 '89673a026-w1-int-1b',
 '89675a049-w1-int-1b',
 '89675a050-w1-int-1b',
 '89679a049-w1-int-1b',
 '89679a050-w1-int-1b',
 '89681a025-w1-int-1b',
 '89681a026-w1-int-1b',
 '89683a049-w1-int-1b',
 '89683a050-w1-int-1b',
 '89685a025-w1-int-1b',
 '89685a026-w1-int-1b',
 '89687a049-w1-int-1b',
 '89687a050-w1-int-1b',
 '89689a025-w1-int-1b',
 '89689a026-w1-int-1b',
 '89691a049-w1-int-1b',
 '89691a050-w1-int-1b',
 '89693a025-w1-int-1b',
 '89693a026-w1-int-1b',
 '89695a049-w1-int-1b',
 '89695a050-w1-int-1b',
 '89697a025-w1-int-1b',
 '89705a026-w1-int-1b',
 '89707a050-w1-int-1b',
 '89709a026-w1-int-1b',
 '89711a050-w1-int-1b',
 '89713a026-w1-int-1b',
 '89715a050-w1-int-1b',
 '89717a026-w1-int-1b',
 '89721a025-w1-int-1b',
 '89725a025-w1-int-1b']

TXT file: /Users/tmzyece25/Desktop/Academia/Physics Data/01) 58.770302 -38.19514/19AugICORE_0001_W2_query_used.tbl.txt
TXT record base names:
['08252r203-w2-int

/var/folders/pf/nh38xgvs0nndkbgjwfbcnmbm0000gp/T/ipykernel_76100/1460263817.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(txt_file, delim_whitespace=True, skiprows=31, header=None, dtype=str)
/var/folders/pf/nh38xgvs0nndkbgjwfbcnmbm0000gp/T/ipykernel_76100/1460263817.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(txt_file, delim_whitespace=True, skiprows=31, header=None, dtype=str)
/var/folders/pf/nh38xgvs0nndkbgjwfbcnmbm0000gp/T/ipykernel_76100/1460263817.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(txt_file, delim_whitespace=True, skiprows=31, header=None, dtype=str)
/var/folders/pf/nh38xgvs0nndkbgjwfbcnmbm0000gp/T/ipykernel_76

TXT record base names:
['13135r050-w1-int-1b',
 '13137r025-w1-int-1b',
 '13139r050-w1-int-1b',
 '13141r026-w1-int-1b',
 '13143r049-w1-int-1b',
 '13143r051-w1-int-1b',
 '13145r026-w1-int-1b',
 '13147r049-w1-int-1b',
 '13147r050-w1-int-1b',
 '13149r025-w1-int-1b',
 '13149r027-w1-int-1b',
 '13159r049-w1-int-1b',
 '13159r050-w1-int-1b',
 '13163r050-w1-int-1b',
 '13165r025-w1-int-1b',
 '13165r026-w1-int-1b',
 '13167r049-w1-int-1b',
 '13167r051-w1-int-1b',
 '13169r026-w1-int-1b',
 '13171r049-w1-int-1b',
 '13171r050-w1-int-1b',
 '13173r025-w1-int-1b',
 '13173r027-w1-int-1b',
 '13175r050-w1-int-1b',
 '13177r026-w1-int-1b',
 '13179r049-w1-int-1b',
 '13179r051-w1-int-1b',
 '13181r026-w1-int-1b']

TXT file: /Users/tmzyece25/Desktop/Academia/Physics Data/01) 58.770302 -38.19514/21AugICORE_0001_W2_query_used.tbl.txt
TXT record base names:
['30614r228-w2-int-1b',
 '30618r228-w2-int-1b',
 '30620r203-w2-int-1b',
 '30620r204-w2-int-1b',
 '30622r227-w2-int-1b',
 '30622r228-w2-int-1b',
 '30628r203-w2-int

/var/folders/pf/nh38xgvs0nndkbgjwfbcnmbm0000gp/T/ipykernel_76100/1460263817.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(txt_file, delim_whitespace=True, skiprows=31, header=None, dtype=str)
/var/folders/pf/nh38xgvs0nndkbgjwfbcnmbm0000gp/T/ipykernel_76100/1460263817.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(txt_file, delim_whitespace=True, skiprows=31, header=None, dtype=str)
/var/folders/pf/nh38xgvs0nndkbgjwfbcnmbm0000gp/T/ipykernel_76100/1460263817.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(txt_file, delim_whitespace=True, skiprows=31, header=None, dtype=str)
/var/folders/pf/nh38xgvs0nndkbgjwfbcnmbm0000gp/T/ipykernel_76

TXT record base names:
['12334b058-w2-int-1b',
 '12338a059-w2-int-1b',
 '12341a035-w2-int-1b',
 '12342a060-w2-int-1b',
 '12350a061-w2-int-1b',
 '12353a037-w2-int-1b',
 '12353a038-w2-int-1b',
 '12354b060-w2-int-1b',
 '12354b061-w2-int-1b',
 '12357a037-w2-int-1b',
 '12358a061-w2-int-1b',
 '12358a062-w2-int-1b',
 '12361a036-w2-int-1b',
 '12361a037-w2-int-1b',
 '12362a060-w2-int-1b',
 '12362a061-w2-int-1b',
 '12366a060-w2-int-1b',
 '12366a061-w2-int-1b',
 '12369a037-w2-int-1b',
 '12370a061-w2-int-1b',
 '12370a062-w2-int-1b',
 '12373a036-w2-int-1b',
 '12373a037-w2-int-1b',
 '12381a036-w2-int-1b',
 '12381a037-w2-int-1b',
 '12382a061-w2-int-1b',
 '12385a036-w2-int-1b',
 '12385a038-w2-int-1b',
 '12386a060-w2-int-1b',
 '12386a061-w2-int-1b',
 '12389a037-w2-int-1b',
 '12390a060-w2-int-1b',
 '12390a062-w2-int-1b',
 '12393a036-w2-int-1b',
 '12393a037-w2-int-1b',
 '12397a037-w2-int-1b',
 '12397a038-w2-int-1b',
 '12398a061-w2-int-1b',
 '12401a037-w2-int-1b']

TXT file: /Users/tmzyece25/Desktop/Acade

/var/folders/pf/nh38xgvs0nndkbgjwfbcnmbm0000gp/T/ipykernel_76100/1460263817.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(txt_file, delim_whitespace=True, skiprows=31, header=None, dtype=str)
/var/folders/pf/nh38xgvs0nndkbgjwfbcnmbm0000gp/T/ipykernel_76100/1460263817.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(txt_file, delim_whitespace=True, skiprows=31, header=None, dtype=str)
/var/folders/pf/nh38xgvs0nndkbgjwfbcnmbm0000gp/T/ipykernel_76100/1460263817.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(txt_file, delim_whitespace=True, skiprows=31, header=None, dtype=str)
/var/folders/pf/nh38xgvs0nndkbgjwfbcnmbm0000gp/T/ipykernel_76